In [2]:
import pandas
import lightfm
from lightfm import LightFM
from lightfm import cross_validation
from scipy.sparse import coo_matrix

C:\Users\Robert\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [3]:
df = pandas.read_csv('write.csv')

In [4]:
topNames = df.groupby("actionName").size().sort_values(ascending=False)[0:50].keys()
df = df[df.actionName.isin(topNames)]
actionByUsers = df.groupby(["userName","actionName"]).size()
actionByUsers=actionByUsers.apply(lambda x:1)
actionByUsers=actionByUsers.unstack()
actionByUsers=actionByUsers.fillna(0.0)
#actionByUsers=actionByUsers.stack()
#users=list(actionByUsers.index.get_level_values(0))
#items=list(actionByUsers.index.get_level_values(1))
#ratings=list(actionByUsers.values)

In [5]:
coo_matrix(actionByUsers.values)

<6115x50 sparse matrix of type '<class 'numpy.float64'>'
	with 25420 stored elements in COOrdinate format>

In [10]:
actionByUsers.columns.values

array(['5G', '5G Dimensioning Guideline',
       '5G Fundamentals - introduction to physical layer',
       '5G downloaded', '5G17 NB Architecture and HW Components Complex',
       '5G18A Essentials', '5GC000275 AirScale Common ASIK',
       '5GC000276 AirScale Capacity ABIL',
       '5GC000353 QSFP+ for Fronthaul LL interface',
       '5GC000608 Supported 5G RAP configurations - III',
       '5GC000623 AirScale Subrack AMIA',
       '5GC000664 AEQD AirScale MAA 64T64R 128AE B43 200W',
       'AirScale System Module in FDD LTE (FL16A)', 'Baseband Pooling',
       'Baseband pooling', 'CORE', 'CORE downloaded', 'Creation date',
       'GSM', 'Inter-site Carrier Aggregation',
       'Introduction to Virtualization and Telco Cloud',
       'IoT - Internet of Things - Introduction', 'LTE', 'LTE FDD',
       'LTE TDD', 'LTE downloaded', 'LTE-M Enhancements I', 'LTE1130',
       'LTE2479', 'LTE2582',
       'LTE2582 DL 4x4 MIMO with Carrier Aggregation', 'LTE2733',
       'LTE2733 Baseband p

In [47]:
#lightfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2, random_state=None)

In [48]:
train, test = cross_validation.random_train_test_split(coo_matrix(actionByUsers.values))

In [49]:
test

<6115x50 sparse matrix of type '<class 'numpy.float64'>'
	with 5084 stored elements in COOrdinate format>

In [50]:
NUM_THREADS = 2
NUM_COMPONENTS = 30
NUM_EPOCHS = 3
ITEM_ALPHA = 1e-6

# Let's fit a WARP model: these generally have the best performance.
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)

# Run 3 epochs and time it.
%time model = model.fit(train,epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

Wall time: 526 ms


In [51]:


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, train, num_threads=NUM_THREADS).mean()
print('AUC: %s' % train_auc)

AUC: 0.94228905


In [52]:
test_auc = auc_score(model,
                    test,
                    train_interactions=train,
                    num_threads=NUM_THREADS).mean()
print('AUC: %s' % test_auc)

AUC: 0.8448011
